In [ ]:
import glob
import json
import os
import subprocess

import geopandas
import pandas
from tqdm.notebook import tqdm

In [ ]:
with open("../config.json") as fh:
    config = json.load(fh)
config

In [ ]:
flood_risk_rasters = glob.glob(os.path.join(
    config['base_path'], 'incoming', 'hazards', 'Maps_Eric', 'flood_hazard', 'ID*'))

flood_risk_rasters

In [ ]:
"Id_future_asda".contains("future")

In [ ]:
for raster_path in flood_risk_rasters:
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
        
    print(name)
    tile_dir = os.path.join(
        config['base_path'], 'data', 'hazards', name, 'tiles')
    
    sp = subprocess.run(["mkdir", "-p", tile_dir], capture_output=True)
    print(sp.stdout.decode('utf-8'))
                         
    sp = subprocess.run(
        ["gdal_retile.py", "-ps", "1024", "1024", "-targetDir", tile_dir, raster_path], 
        capture_output=True)
    print(sp.stdout.decode('utf-8'))

In [ ]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    tile_dir = os.path.join(config['base_path'], 'data', 'hazards', name, 'tiles')
    
    threshold_dir = os.path.normpath(os.path.join(tile_dir, '..', 'thresholds'))
    subprocess.run(["mkdir", "-p", threshold_dir])
                   
    vector_dir = os.path.normpath(os.path.join(tile_dir, '..', 'polygons'))
    subprocess.run(["mkdir", "-p", vector_dir])
    
    for tile_path in glob.glob(tile_dir + '/*.tif'):
        tile_name = os.path.basename(tile_path).replace(".tif", "")
        
        # thresholds = [0.0, 0.25, 0.5, 1.0, 2.0, 3.0, 5.0, 10.0, 999.0]
        thresholds = [3,5,7]
        
        for min_depth, max_depth in zip(thresholds, thresholds[1:]):    
            # Calculate mask where raster values are between thresholds
            sp = subprocess.run([
                "gdal_calc.py",
                "-A", 
                f"{tile_dir}/{tile_name}.tif",
                f"--outfile={threshold_dir}/{tile_name}_{min_depth}m{max_depth}m.tif",
                f"--calc=logical_and(A>={min_depth},A<{max_depth})",
                "--format=GTiff",
                "--type=Byte",
                "--NoDataValue=0",
                "--co=SPARSE_OK=YES",
                "--quiet",
                "--co=COMPRESS=LZW",
            ], check=True)

            # Convert raster masks to vector polygons
            sp = subprocess.run([
                "gdal_polygonize.py",
                f"{threshold_dir}/{tile_name}_{min_depth}m{max_depth}m.tif",
                "-q",
                "-f", 
                "GPKG",
                f"{vector_dir}/{tile_name}_{min_depth}m{max_depth}m.gpkg"
            ], check=True)

In [ ]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    vector_dir = os.path.join(config['base_path'], 'data', 'hazards', name, 'polygons')
    
    for min_depth, max_depth in zip(thresholds, thresholds[1:]):    
        match = os.path.join(vector_dir, f"*{min_depth}m{max_depth}m.gpkg")
        tiles = glob.glob(match)
        
        dfs = []
        for tile in tqdm(tiles):
            df = geopandas.read_file(tile)
            dfs.append(df)
            
        df = pandas.concat(dfs)
        df.to_file(
            os.path.join(config['base_path'], 'data', 'hazards', f"{name}_{min_depth}-{max_depth - 1}.gpkg"),
            driver="GPKG"
        )
    

In [ ]:
for raster_path in flood_risk_rasters:    
    if "future" in os.path.basename(raster_path):
        name = "future"
    else:
        name = "current"
    
    # Clean up - remove working directories
    working_dir = os.path.join(config['base_path'], 'data', 'hazards', name)
    subprocess.run(["rm", "-r", working_dir])

# Drought Maps

In [ ]:
basepath = 'C:\\Users\\Sarah\\Nexus365\\Tom Russell - ghana-oxford-infrastructure-adaptation-2020'

## Drought Hazard Map Eric 2010

In [ ]:
drought2010_path = os.path.join(basepath, 'incoming/hazards/Maps_Eric/hazard_drought_2010.shp')
GHA_drought2010 = geopandas.read_file(drought2010_path)

In [ ]:
GHA_drought2010 = GHA_drought2010.to_crs("EPSG:32630")

In [ ]:
GHA_drought2010 = GHA_drought2010[['DN','geometry']]
GHA_drought2010 = GHA_drought2010.reset_index()
GHA_drought2010 = GHA_drought2010.rename(columns = {'index':'ID', 'DN':'HAZARD_LEVEL'})

In [ ]:
GHA_drought2010.head()

In [ ]:
GHA_drought2010.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2010.gpkg'), layer='drought2010', driver="GPKG")

#### Create extra layer for each Hazard level

In [ ]:
GHA_drought2010_1 = GHA_drought2010[(GHA_drought2010.HAZARD_LEVEL == 1)]

In [ ]:
GHA_drought2010_1 = GHA_drought2010_1[['HAZARD_LEVEL','geometry']]
GHA_drought2010_1 = GHA_drought2010_1.reset_index()

In [ ]:
GHA_drought2010_1.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2010_1.gpkg'), layer='drought2010_1', driver="GPKG")

In [ ]:
GHA_drought2010_2 = GHA_drought2010[(GHA_drought2010.HAZARD_LEVEL == 2)]

In [ ]:
GHA_drought2010_2 = GHA_drought2010_2[['HAZARD_LEVEL','geometry']]
GHA_drought2010_2 = GHA_drought2010_2.reset_index()

In [ ]:
GHA_drought2010_2.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2010_2.gpkg'), layer='drought2010_2', driver="GPKG")

In [ ]:
GHA_drought2010_3 = GHA_drought2010[(GHA_drought2010.HAZARD_LEVEL == 3)]

In [ ]:
GHA_drought2010_3 = GHA_drought2010_3[['HAZARD_LEVEL','geometry']]
GHA_drought2010_3 = GHA_drought2010_3.reset_index()

In [ ]:
GHA_drought2010_3.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2010_3.gpkg'), layer='drought2010_3', driver="GPKG")

## Drought Hazard Map Eric 2050

In [ ]:
drought2050_path = os.path.join(basepath, 'incoming/hazards/Maps_Eric/hazard_drought_2050.shp')
GHA_drought2050 = geopandas.read_file(drought2050_path)

In [ ]:
GHA_drought2050 = GHA_drought2050.to_crs("EPSG:32630")

In [ ]:
GHA_drought2050 = GHA_drought2050.reset_index()

In [ ]:
GHA_drought2050 = GHA_drought2050[['index','DN','geometry']]
GHA_drought2050 = GHA_drought2050.rename(columns = {'index':'ID', 'DN':'HAZARD_LEVEL'})

In [ ]:
GHA_drought2050.head()

In [ ]:
GHA_drought2050.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2050.gpkg'), layer='drought2050', driver="GPKG")

#### Create extra layer for each Hazard level

In [ ]:
GHA_drought2050_1 = GHA_drought2050[(GHA_drought2050.HAZARD_LEVEL == 1)]

In [ ]:
GHA_drought2050_1 = GHA_drought2050_1[['HAZARD_LEVEL','geometry']]
GHA_drought2050_1 = GHA_drought2050_1.reset_index()

In [ ]:
GHA_drought2050_1.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2050_1.gpkg'), layer='drought2050_1', driver="GPKG")

In [ ]:
GHA_drought2050_2 = GHA_drought2050[(GHA_drought2050.HAZARD_LEVEL == 2)]

In [ ]:
GHA_drought2050_2 = GHA_drought2050_2[['HAZARD_LEVEL','geometry']]
GHA_drought2050_2 = GHA_drought2050_2.reset_index()

In [ ]:
GHA_drought2050_2.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2050_2.gpkg'), layer='drought2050_2', driver="GPKG")

In [ ]:
GHA_drought2050_3 = GHA_drought2050[(GHA_drought2050.HAZARD_LEVEL == 3)]

In [ ]:
GHA_drought2050_3 = GHA_drought2050_3[['HAZARD_LEVEL','geometry']]
GHA_drought2050_3 = GHA_drought2050_3.reset_index()

In [ ]:
GHA_drought2050_3.to_file(os.path.join(
basepath,'data','hazards','drought','GHA_drought2050_3.gpkg'), layer='drought2050_3', driver="GPKG")

# Landslide Map

## Landslide Hazard Data - ID81 GH_landslide hazard_VictorAddabor

In [ ]:
landslide_path = os.path.join(basepath, 'incoming/hazards/Landslides/ID81 GH_landslide hazard_VictorAddabor/landslide.shp')
GHA_landslide = geopandas.read_file(landslide_path)

In [ ]:
GHA_landslide = GHA_landslide.set_crs("EPSG:32630")

In [ ]:
GHA_landslide = GHA_landslide[['HEIGHT','SLIDE','geometry']]
GHA_landslide = GHA_landslide.reset_index()
GHA_landslide = GHA_landslide.rename(columns = {'index':'ID', 'SLIDE':'HAZARD_LEVEL'})

In [ ]:
GHA_landslide.head()

In [ ]:
GHA_landslide.to_file(os.path.join(
basepath,'data','hazards','GHA_landslides.gpkg'), layer='landslides', driver="GPKG")

#### Create extra layer for each Hazard level

In [ ]:
GHA_landslide_1 = GHA_landslide[(GHA_landslide.HAZARD_LEVEL == 'LOW')]

In [ ]:
GHA_landslide_1 = GHA_landslide_1[['HEIGHT','HAZARD_LEVEL','geometry']]
GHA_landslide_1 = GHA_landslide_1.reset_index()

In [ ]:
GHA_landslide_1.to_file(os.path.join(
basepath,'data','hazards','landslides','GHA_landslides_1.gpkg'), layer='landslides_1', driver="GPKG")

In [ ]:
GHA_landslide_2 = GHA_landslide[(GHA_landslide.HAZARD_LEVEL == 'MODERATE')]

In [ ]:
GHA_landslide_2 = GHA_landslide_2[['HEIGHT','HAZARD_LEVEL','geometry']]
GHA_landslide_2 = GHA_landslide_2.reset_index()

In [ ]:
GHA_landslide_2.to_file(os.path.join(
basepath,'data','hazards','landslides','GHA_landslides_2.gpkg'), layer='landslides_2', driver="GPKG")

In [ ]:
GHA_landslide_3 = GHA_landslide[(GHA_landslide.HAZARD_LEVEL == 'VERY HIGH')]

In [ ]:
GHA_landslide_3 = GHA_landslide_3[['HEIGHT','HAZARD_LEVEL','geometry']]
GHA_landslide_3 = GHA_landslide_3.reset_index()

In [ ]:
GHA_landslide_3.to_file(os.path.join(
basepath,'data','hazards','landslides','GHA_landslides_3.gpkg'), layer='landslides_3', driver="GPKG")